### Riddler Classic

Another week, another dice-y Classic!

When you roll a pair of fair dice, the most likely outcome is 7 (which occurs 1/6 of the time) and the least likely outcomes are 2 and 12 (which each occur 1/36 of the time).

Annoyed by the variance of these probabilities, I set out to create a pair of “uniform dice.” These dice still have sides that are uniquely numbered from 1 to 6, and they are identical to each other. However, they are weighted so that their sum is more uniformly distributed between 2 and 12 than that of fair dice.

Unfortunately, it is impossible to create a pair of such dice so that the probabilities of all 11 sums from 2 to 12 are identical (i.e., they are all 1/11). But I bet we can get pretty close.

The variance of the 11 probabilities is the average value of the squared difference between each probability and the average probability (which is, again, 1/11). One way to make my dice as uniform as possible is to minimize this variance.

So how should I make my dice as uniform as possible? In other words, which specific weighting of the dice minimizes the variance among the 11 probabilities? That is, what should the probabilities be for rolling 1, 2, 3, 4, 5 or 6 with one of the dice?

In [1]:
from dataclasses import dataclass, field
from itertools import permutations, combinations, groupby

In [2]:
@dataclass(frozen=True)
class proper_die:
    sides = [n + 1 for n in range(6)]
    prob: list = field(default_factory=lambda: [1/6 for n in range(6)])
        
    def build(self):
        return [(s, p) for (s,p) in zip(self.sides, self.prob)]
        
    def build_distribution(self):
        """Build out all possible scores"""
        c = self.build()
        
        return [(x[0] + y[0], x[1] * y[1]) for x in c for y in c]
    
    def build_final(self):
        """Sum up probabilities"""
        output = self.build_distribution()
        output.sort(key = lambda x: x[0]) # sort by value from double roll
        final_dict = {}
        for key, rows in groupby(output, lambda x: x[0]): # groupby first element of tuple & sum
            final_dict[key] = sum(r[1] for r in rows)
        return final_dict

In [3]:
proper_die([1/6 for n in range(6)]).build_final()

{2: 0.027777777777777776,
 3: 0.05555555555555555,
 4: 0.08333333333333333,
 5: 0.1111111111111111,
 6: 0.1388888888888889,
 7: 0.16666666666666669,
 8: 0.1388888888888889,
 9: 0.1111111111111111,
 10: 0.08333333333333333,
 11: 0.05555555555555555,
 12: 0.027777777777777776}

### Process of Solving: 

We now have a nice class where we can pass in an ordered vector representing probabilities of different rolls. 

Next we can start experimenting. 

In [4]:
new_prob = [1/5] + [.15] * 4 + [1/5]
assert(sum(new_prob) == 1)
proper_die(new_prob).build_final()

{2: 0.04000000000000001,
 3: 0.06,
 4: 0.08249999999999999,
 5: 0.105,
 6: 0.1275,
 7: 0.16999999999999998,
 8: 0.1275,
 9: 0.105,
 10: 0.08249999999999999,
 11: 0.06,
 12: 0.04000000000000001}

### Above Is Tedious

We can instead calculate random numbers by the following:

- choose 3 random numbers (1,5,7)
- sum up (13)
- divide each number by sum * 2 ([0.23304853 0.10251759 0.16443388]) -> sums to 0.5

We then just reverse and add up

We can then calculate variance & return lowest possible

In [5]:
import numpy as np
a = np.random.random(3)
a /= a.sum() * 2
print(a)
sum(a)

[0.31275073 0.02597501 0.16127427]


0.5000000000000001

In [6]:
list(a) + list(a[::-1])

[0.3127507286681019,
 0.02597500592646211,
 0.16127426540543607,
 0.16127426540543607,
 0.02597500592646211,
 0.3127507286681019]

In [7]:
def random_probs(n=3):
    a = np.random.random(n)
    a /= a.sum() * 2
    return list(a) + list(a[::-1])

assert(sum(random_probs()) == 1)

random_probs()

[0.0898215624572943,
 0.20039860766776663,
 0.2097798298749391,
 0.2097798298749391,
 0.20039860766776663,
 0.0898215624572943]

In [8]:
def variance(my_dict):
    """Pass in vector of 11 & solve for variance"""
    return sum([(x - (1/11))**2 for x in my_dict.values()]) / len(my_dict.values())

In [9]:
test = proper_die([1/6 for n in range(6)]).build_final()
variance(test)

0.0019768390980512197

### Big Sim

Should have written it all in numpy for speed. Next time. 

In [10]:
min_var = variance(test)
min_prob = [1/6 for n in range(6)]
for _ in range(1_000_000):
    
    # run random prob
    prob_v = random_probs()
    outcome = proper_die(prob_v).build_final()
    var = variance(outcome)
    
    # see if its less
    if var < min_var:
        #print(f"Found a lower var on step {_}")
        min_var = var
        min_prob = prob_v

In [11]:
print(f"Smallest variance was {min_var:.8f}")
print(f"Using the following probs: {min_prob}")

Smallest variance was 0.00121759
Using the following probs: [0.24391792595970044, 0.1373532511267015, 0.11872882291359811, 0.11872882291359811, 0.1373532511267015, 0.24391792595970044]
